# Comparing concentrations plasma eider 2021

Here we test whether there are statistically significant differences in concentration between early and late incubation within a group (early and late breeders) and between early incubation and late incubation across breeding season. We also test if there is a difference in clutch size (called "Eggs" in the script) between the early and late breeders (using the clutch size recorded during our first sampling). 

In [ ]:
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import levene
import statsmodels.stats.multitest as ssm
import statistics

## Read in file

In [ ]:
# Read in data file - use the one with LODs replaced

allCECs = pd.read_csv('Appropriate file path to plasmaEiderNTNU2021WithLODs.csv')

compoundList = ["BPA", "BPS", "BzP3", "mMP", "mEP", "Eggs"]

## Divide dataframe into groups

In [ ]:
# Divide into groups (early and late incubation, early and late breeders)

# Set dataframe
df = allCECs

# Early incubation, early & late breeding period
earlyInc = df[(df.Incubation == 'early') & (df.BreedingPeriod != 'mid')].reset_index(drop=True)

# Late incubation, early & late breeding period
lateInc = df[(df.Incubation == 'late') & (df.BreedingPeriod != 'mid')].reset_index(drop=True)

# Early breeders
earlyBreeders = df[(df.BreedingPeriod == 'early')].reset_index(drop=True)

# Late breeders
lateBreeders = df[(df.BreedingPeriod == 'late')].reset_index(drop=True)

# Early incubation, early breeding period
earlyIncEarlyBP = df[(df.Incubation == 'early') & (df.BreedingPeriod == 'early')].reset_index(drop=True)

# Early incubation, late breeding period
earlyIncLateBP = df[(df.Incubation == 'early') & (df.BreedingPeriod == 'late')].reset_index(drop=True)

# Late incubation, early breeding period
lateIncEarlyBP = df[(df.Incubation == 'late') & (df.BreedingPeriod == 'early')].reset_index(drop=True)

# Late incubation, late breeding period
lateIncLateBP = df[(df.Incubation == 'late') & (df.BreedingPeriod == 'late')].reset_index(drop=True)

## Test normality and variance

In [ ]:
sampleOne = allCECs[(allCECs.PlasmaSample == 1) & (allCECs.BreedingPeriod != 'mid')].reset_index(drop=True)
sampleTwo = allCECs[(allCECs.PlasmaSample == 2) & (allCECs.BreedingPeriod != 'mid')].reset_index(drop=True)

sampleOneEarlyBreeders = sampleOne[(sampleOne.BreedingPeriod == 'early')].reset_index(drop=True)
sampleTwoEarlyBreeders = sampleTwo[(sampleTwo.BreedingPeriod == 'early')].reset_index(drop=True)

sampleOneLateBreeders = sampleOne[(sampleOne.BreedingPeriod == 'late')].reset_index(drop=True)
sampleTwoLateBreeders = sampleTwo[(sampleTwo.BreedingPeriod == 'late')].reset_index(drop=True)

In [ ]:
# Early breeders, T1 vs T2:
for compound in compoundList:
    difference = []
    for i in range(len(sampleOneEarlyBreeders)):
        diff = (sampleOneEarlyBreeders[compound][i] - sampleTwoEarlyBreeders[compound][i])
        difference.append(diff)
    if ((round(scipy.stats.shapiro(difference)[1], 2) >= 0.05) & (round(levene(sampleOneEarlyBreeders[compound], sampleTwoEarlyBreeders[compound])[1], 2) >= 0.05)):
        print(f"{compound} is likely normally distributed with equal variance between T1 and T2 for early breeders.")
        
# Late breeders, T1 vs T2:
for compound in compoundList:
    difference = []
    for i in range(len(sampleOneLateBreeders)):
        diff = (sampleOneLateBreeders[compound][i] - sampleTwoLateBreeders[compound][i])
        difference.append(diff)
    if ((round(scipy.stats.shapiro(difference)[1], 2) >= 0.05) & (round(levene(sampleOneLateBreeders[compound], sampleTwoLateBreeders[compound])[1], 2) >= 0.05)):
        print(f"{compound} is likely normally distributed with equal variance between T1 and T2 for late breeders.")
        
# Early incubation (sample 1/T1):
for compound in compoundList:
    if ((round(scipy.stats.shapiro(sampleOne[compound])[1], 2) >= 0.05) & (round(levene(sampleOneEarlyBreeders[compound], sampleOneLateBreeders[compound])[1], 2) >= 0.05)):
        print(f"{compound} is likely normally distributed with equal variance between T1 samples for early and late breeders.")
        
# Late incubation (sample 2/T2):
for compound in compoundList:
    if ((round(scipy.stats.shapiro(sampleTwo[compound])[1], 2) >= 0.05) & (round(levene(sampleTwoEarlyBreeders[compound], sampleTwoLateBreeders[compound])[1], 2) >= 0.05)):
        print(f"{compound} is likely normally distributed with equal variance between T2 samples for early and late breeders.")

## Test concentration differences

Based on the Shapiro-Wilk and Levene results: 

Test BPA differences between paired samples for early and late breeders using parametric test (note that BzP-3 was not above the 60% cut-off for early breeders, so it won't be tested). 

All other compounds: non-parametric tests.

Clutch size at early incubation was normally distributed with equal variance, so test that with parametric test.

### Parametric tests

In [ ]:
# Compare incubation stages
# Use parametric paired t-test since the data are paired

# Only BPA meet test assumptions
compoundList = ["BPA"]

# Early breeders

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = scipy.stats.ttest_rel(earlyIncEarlyBP[compound], lateIncEarlyBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("paired t-test")
    compoundName.append(compound)
    breedingPeriod.append("Early")
    incubation.append("Both")
    
paraCompareIncEarlyBP = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])


# Late breeders

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = scipy.stats.ttest_rel(earlyIncLateBP[compound], lateIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("paired t-test")
    compoundName.append(compound)
    breedingPeriod.append("Late")
    incubation.append("Both")

paraCompareIncLateBP = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

#### Clutch size

In [ ]:
# Compare clutch size at early incubation between breeding periods

# Compound list with only eggs
compoundList = ["Eggs"]

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = scipy.stats.ttest_ind(earlyIncEarlyBP[compound], earlyIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("t-test")
    compoundName.append(compound)
    breedingPeriod.append("Both")
    incubation.append("Early")
    
compareBPEggs = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

# Late incubation comparison
# Not tested

### Non-parametric tests

In [ ]:
# Compare incubation stages
# Use the non-parametric Wilcoxon test since the data are paired

# Do not include BPA as a parametric test was used for that compound
compoundList = ["BPS", "BzP3", "mMP", "mEP"]

# Early breeders
# No non-parametric test done for comparing paired samples for early breeders as only BPA passed the 60% threshold

# Late breeders
# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = stats.wilcoxon(earlyIncLateBP[compound], lateIncLateBP[compound], zero_method="zsplit", correction=False)[1]
    pvalues.append(pvalue)
    statisticalTest.append("Wilcoxon")
    compoundName.append(compound)
    breedingPeriod.append("Late")
    incubation.append("Both")
    
compareIncLateBP = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

In [ ]:
# Compare breeding periods with Mann-Whitney U test

# Early incubation comparison

# Compound list without mMP and mEP and no BPS as it was not above 60% at T1 for either early nor late breeders
compoundList = ["BPA", "BzP3"]

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = stats.mannwhitneyu(earlyIncEarlyBP[compound], earlyIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("Mann-Whitney")
    compoundName.append(compound)
    breedingPeriod.append("Both")
    incubation.append("Early")
    
compareBPEarlyInc = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

# Late incubation comparison
# Compound list without mMP and mEP
compoundList = ["BPA", "BPS", "BzP3"]

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = stats.mannwhitneyu(lateIncEarlyBP[compound], lateIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("Mann-Whitney")
    compoundName.append(compound)
    breedingPeriod.append("Both")
    incubation.append("Late")
    
compareBPLateInc = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

In [ ]:
# Remove NaN values

earlyIncLateBP = earlyIncLateBP.dropna()
lateIncLateBP = lateIncLateBP.dropna()

In [ ]:
# Compare breeding periods with Mann-Whitney U test
# Early incubation comparison

# Compound list with only mMP and mEP
compoundList = ["mMP", "mEP"]

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = stats.mannwhitneyu(earlyIncEarlyBP[compound], earlyIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("Mann-Whitney")
    compoundName.append(compound)
    breedingPeriod.append("Both")
    incubation.append("Early")
    
compareBPEarlyIncPMs = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

# Late incubation comparison

# Create empty vectors to store the values from the for loop
pvalues = []
statisticalTest = []
compoundName = []
breedingPeriod = []
incubation = []

for compound in compoundList:
    pvalue = stats.mannwhitneyu(lateIncEarlyBP[compound], lateIncLateBP[compound])[1]
    pvalues.append(pvalue)
    statisticalTest.append("Mann-Whitney")
    compoundName.append(compound)
    breedingPeriod.append("Both")
    incubation.append("Late")
    
compareBPLateIncPMs = pd.DataFrame(zip(compoundName, pvalues, breedingPeriod, incubation, statisticalTest), 
                                 columns=["compound", "pvalue", "breedingPeriod", "incubation", "statisticalTest"])

In [ ]:
# Merge all the dataframes
statTests = pd.concat([paraCompareIncEarlyBP, paraCompareIncLateBP, compareIncLateBP, compareBPEarlyInc, compareBPLateInc, compareBPEarlyIncPMs, compareBPLateIncPMs, compareBPEggs])

# Add column with adjusted p-values
statTests["adjustedPValue"] = ssm.multipletests(statTests.pvalue, method="fdr_bh")[1]

statTests